In [1]:
import os
import pandas as pd

In [2]:
# Sciprt to get raw data.
exec( open( "../raw_data/get_raw_data.py" ).read() )

In [3]:
# Get the raw data using the function `get_raw_data`
# from the above script.
relative_path = "../raw_data"

app_list     = get_raw_data( "app_list", relative_path )
app_data     = get_raw_data( "app_data", relative_path )
app_reviews  = get_raw_data( "app_reviews", relative_path )
user_reviews = get_raw_data( "user_reviews", relative_path )

cleaned_app_list     = None
cleaned_app_data     = None
cleaned_app_reviews  = None
cleaned_user_reviews = None

In [4]:
# Start cleaning

# 1 - `app_list` data does not require any cleaning,
#     it is simpyl a mapping between app_ids and 
#     app_names.
cleaned_app_list = app_list

# debug
cleaned_app_list

,app_id,app_title
0,5,Dedicated Server
1,7,Steam Client
2,8,winui2
3,10,Counter-Strike
4,20,Team Fortress Classic
...,...,...
104005,1456550,The Tower Of TigerQiuQiu Soapbubble
104006,1456850,Solicitude Wake-up Demo
104007,1457260,Masters of Puzzle - Halloween Edition: Undeadl...
104008,1457270,Masters of Puzzle - Halloween Edition: Pumpkin...


In [5]:
# 2 - `app_data` contains some invalid values as we can see above.
#     This is because some apps are for testing or reserved for
#     internal usage. Fortunately, Steam API return a False status
#     for invalid apps, So, we are going to remove the rows with status
#     equals to False.
cleaned_app_data = app_data[app_data.status != False]

#     Now we can remove the status column since all the values 
#     are True.
cleaned_app_data = cleaned_app_data.drop( columns = ["status"] )

#     We are also going to remove applications which do not have a
#     `game` type. Other types such as trailer, dlc, demo, mod etc.
#     are not of any use to us.
cleaned_app_data = cleaned_app_data[cleaned_app_data.type == "game"]

#     Now we can also remove the type column since all the values 
#     are `game`.
cleaned_app_data = cleaned_app_data.drop( columns = ["type"] )

#     debug
cleaned_app_data

,app_id,name,required_age,platforms,metacritic_score,category_ids,categories,genre_ids,genres,recommendations
3,10,Counter-Strike,0.0,windows:mac:linux,88.0,1:49:36:37:8,Multi-player:PvP:Online PvP:Shared/Split Scree...,1,Action,94916.0
4,20,Team Fortress Classic,0.0,windows:mac:linux,NaN,1:49:36:37:8:44,Multi-player:PvP:Online PvP:Shared/Split Scree...,1,Action,3597.0
5,30,Day of Defeat,0.0,windows:mac:linux,79.0,1:8,Multi-player:Valve Anti-Cheat enabled,1,Action,2669.0
6,40,Deathmatch Classic,0.0,windows:mac:linux,NaN,1:49:36:37:8:44,Multi-player:PvP:Online PvP:Shared/Split Scree...,1,Action,1244.0
7,50,Half-Life: Opposing Force,0.0,windows:mac:linux,NaN,2:1:8:44,Single-player:Multi-player:Valve Anti-Cheat en...,1,Action,7560.0
...,...,...,...,...,...,...,...,...,...,...
103979,1455060,TERMINUS,0.0,windows,NaN,2,Single-player,4:23,Casual:Indie,NaN
103980,1455090,Good puzzle: Castles,0.0,windows,NaN,2:22,Single-player:Steam Achievements,4:23:28,Casual:Indie:Simulation,NaN
103983,1455420,Choco Pixel S,0.0,windows,NaN,2:22:23,Single-player:Steam Achievements:Steam Cloud,4:23,Casual:Indie,NaN
103984,1455430,Harvest Green,0.0,windows,NaN,2,Single-player,25:4:23:3:28:2,Adventure:Casual:Indie:RPG:Simulation:Strategy,NaN


In [6]:
# 3 - Similar to `app_data`, `app_reviews` some  invalid values,
#     we are going to discard those rows and remove the `status`
#     column.
cleaned_app_reviews = app_reviews[app_reviews.status != False]

#     Remove the `status` column
cleaned_app_reviews = cleaned_app_reviews.drop( columns = ["status"] )

#     Since many apps have no reviews we are also going to remove them.
cleaned_app_reviews = cleaned_app_reviews[cleaned_app_reviews.total_reviews != 0]

#     debug
cleaned_app_reviews

,app_id,review_score,review_score_desc,total_positive,total_negative,total_reviews
3,10,9,Overwhelmingly Positive,19908,753,20661
4,20,8,Very Positive,1889,385,2274
5,30,8,Very Positive,1169,169,1338
6,40,8,Very Positive,548,131,679
7,50,9,Overwhelmingly Positive,4047,192,4239
...,...,...,...,...,...,...
103877,1450220,5,Mixed,11,6,17
103894,1450700,0,3 user reviews,3,0,3
103926,1452150,0,3 user reviews,3,0,3
103981,1454920,0,3 user reviews,3,0,3


In [7]:
def total_hours( x ):
    return x.sum()

def num_users( x ):
    return len( x.index )

# Group all the reviews for any sepcific application and sum
# their hours and total number of users.
cleaned_user_reviews = user_reviews.groupby( ["app_id"] )["playtime_forever"].agg( [ num_users, total_hours ] ).reset_index()

# debug
cleaned_user_reviews

,app_id,num_users,total_hours
0,10,1000.0,19124629.0
1,20,1000.0,6349770.0
2,30,1000.0,7296999.0
3,40,679.0,1374408.0
4,50,1000.0,1406570.0
...,...,...,...
48980,1450220,17.0,0.0
48981,1450700,3.0,0.0
48982,1452150,3.0,0.0
48983,1454920,3.0,0.0


In [8]:
# Save the cleaned data.

if not os.path.exists( "../cleaned_data" ):
    os.makedirs( "../cleaned_data" )
    
cleaned_app_data.to_csv( "../cleaned_data/app_data.csv", index = False )
cleaned_app_reviews.to_csv( "../cleaned_data/app_reviews.csv", index = False )
cleaned_user_reviews.to_csv( "../cleaned_data/user_data.csv", index = False )